## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
print(df.head(10))

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

Посмотрим также последние 10 строк:

In [2]:
print(df.tail(10))

       children  days_employed  dob_years       education  education_id  \
21515         1    -467.685130         28         среднее             1   
21516         0    -914.391429         42          высшее             0   
21517         0    -404.679034         42          высшее             0   
21518         0  373995.710838         59         СРЕДНЕЕ             1   
21519         1   -2351.431934         37  ученая степень             4   
21520         1   -4529.316663         43         среднее             1   
21521         0  343937.404131         67         среднее             1   
21522         1   -2113.346888         38         среднее             1   
21523         3   -3112.481705         38         среднее             1   
21524         2   -1984.507589         40         среднее             1   

          family_status  family_status_id gender income_type  debt  \
21515   женат / замужем                 0      F   сотрудник     1   
21516   женат / замужем           

Ознакомимся с общей информацией:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


И сводной информацией:

In [4]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод


При наглядном анализе выявила:

1) минимальное значение детей -1
2) в столбце стажа:
    -отрицательные значения;
    -аномально высокие(300000дней  - это больше 800 лет)
    -есть пропуски.
3) в столбце образование данные указаны разным регистром;
5) в столбце целей несколько формулировок одного и того же. Возможно, они прописаны вручную. Было бы оптимально оптимизировать выбором из установленных вариантов
6) пропущены значения в столбцах стажа и заработной платы. Наверняка эти значения находятся в одной строке, так как тут прямая связь.



### Шаг 2. Предобработка данных

### Обработка пропусков

Проверим общее количество пропусков:

In [5]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Видим пропуски только в 2 столбах. Рассмотрим их более подробно

In [6]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Отрицательных значений слишком весомое количество, то есть практически все. Удалить мы их не можем, примем за факт, что здесь опечатка, использован знак "-" как маркировка значения. Переведем все значения к модулю.

In [7]:
df['days_employed'] = df['days_employed'].abs()

In [8]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [9]:
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

От отрицательных значений избавились. Перейдем к максимальным. Необходимо проверить, сколько таких данных - для этого возьмем самого великовозрастного клиента, чтобы по нему определить потолок дней:

In [10]:
max_years = df['dob_years'].max()
max_years

75

Предположим, что он работал с 14 лет. Найдем максимальное количество дней работы:

In [11]:
max_days_employment = (max_years - 14) * 365
max_days_employment


22265

In [12]:
(df[df['days_employed'] > max_days_employment]['days_employed']).count()


3445

Получилось большое количество данных. Предположим, что такие значения возникли при неверном подсчете дней - например, посчитали часы работы. разделим эти данные на 24 и произведем замену

In [13]:
new_employed = (df[df['days_employed'] > max_days_employment]['days_employed']) / 24


In [14]:
df.loc[df['days_employed'] > 22265, 'days_employed']= new_employed

In [15]:
df['days_employed'].describe()

count    19351.000000
mean      4641.641176
std       5355.964289
min         24.141633
25%        927.009265
50%       2194.220567
75%       5537.882441
max      18388.949901
Name: days_employed, dtype: float64

Так лучше. Теперь найдем количество пропусков

In [16]:
df['days_employed'].isnull().sum()

2174

Если заменить их на 0, то потеряем большой массив данных(почти 10%), заменить средним значением или медианой тоже не верно, как как значение может быть выше возраста. Так как количество дней занятости так или иначе зависит от возраста, то сгруппируем клиентов по возрасту, найдем для каждой категории среднее значение дней работы и заменим пропущенные значения на него.
По поводу того, что у нас есть категории безработных и в декрете - они также могли работать ранее.

In [17]:
df['dob_years'].quantile([0.25,0.5,0.75])


0.25    33.0
0.50    42.0
0.75    53.0
Name: dob_years, dtype: float64

до 33 молодежь, до 42 взрослые, до 53 предпинсионеры, от 53 пенсионеры

In [18]:
def category_years(dob_years):
    if dob_years < 33 :
        return 'молодежь'
    if dob_years <= 42:
        return 'взрослые'
    if dob_years <= 53:
        return 'предпенсионеры'
    return 'пенсионеры'
df['category_years'] = df['dob_years'].apply(category_years)
df.groupby('category_years')['days_employed'].mean()

category_years
взрослые           2388.017018
молодежь           1471.645312
пенсионеры        10760.722834
предпенсионеры     4034.096358
Name: days_employed, dtype: float64

Избавимся от пропущенных значений. Заменяю на целочисленные значения без цифр после запятой, так как это дни, а десятые части дня незначительны для стажа.
(#поясню, почему я сначала заменила на 0 - у меня не получалось заменить, если приравнивать к пропущенным значениям. а так как у нас в этом столбце минимальное больше 0, то я решила, что там можно. И метод fillna не использовала не тут, не выше, потому что тогда меняется данные в этом масиве, но почему то в полной таблице нет. в слэк у всех такая проблема, пока дружно над ней думали, решила сделать топорно и получилось) 

In [20]:
df['days_employed'] = df['days_employed'].fillna(1)
df.loc[(df['category_years']=='пенсионеры') & (df['days_employed'] == 0),'days_employed'] = 10760
df.loc[(df['category_years']=='взрослые') & (df['days_employed'] == 0),'days_employed'] = 2388
df.loc[(df['category_years']=='молодежь') & (df['days_employed'] == 0),'days_employed'] = 1471
df.loc[(df['category_years']=='предпенсионеры') & (df['days_employed'] == 0),'days_employed'] = 4034

In [21]:
df['days_employed'].describe()

count    21525.000000
mean      4172.941807
std       5267.296063
min          1.000000
25%        610.652074
50%       1808.053434
75%       4779.587738
max      18388.949901
Name: days_employed, dtype: float64

Со столбцом дней стажа разобрались - нет пропусков, нет отрицательных чисел и нет завышенных значений.
У нас остались пропуски в доходе.

In [23]:
df['total_income'].min()

20667.26379327158

In [24]:
df['total_income'].max()

2265604.028722744

Помимо пропусков, проверила, есть ли отрицательные или 0 значения. Судя по минимуму, тут только пропуски. И так как тип Float, то мы имеем дело с числами. Опять же, приравнять доход всем средний не можем - слишком велика разница. разделим снова по категориям, только уже по типу занятости. Сгруппируем доход по типу занятости и найдем среднюю для каждого дохода

In [25]:
df.groupby('income_type')['total_income'].mean()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        170898.309923
компаньон          202417.461462
пенсионер          137127.465690
предприниматель    499163.144947
сотрудник          161380.260488
студент             98201.625314
Name: total_income, dtype: float64

наша максимальная цифра по всем показателям и средняя по тем же предпринимателям сильно разнится. Проверим, сколько у нас значений выше средней по предпринимателям

In [26]:
df[df['total_income'] > 499163]['total_income'].count()

225

Получили число клиентов около 1%. Так как у нас нет привязки к валюте и мы не знаем, насколько число 2 млн большое, не буду его трогать. Но могу предположить, что возникло оно, оттого, что люди вместо дохода в месяц написали за год или за пол года, допустим. С учетом того, что средняя по всем категориям логичная (Безработный может получать субсидии, доход с акций, паевых фондов, пособий, студент повышенную степендию). Поэтому оставим, как есть, заменим только пропуски на среднюю

In [27]:
df['total_income'] = df['total_income'].fillna(1)
df.loc[(df['income_type']=='безработный') & (df['total_income'] == 0),'total_income'] = 131339
df.loc[(df['income_type']=='в декрете') & (df['total_income'] == 0),'total_income'] = 53829
df.loc[(df['income_type']=='госслужащий') & (df['total_income'] == 0),'total_income'] = 170898
df.loc[(df['income_type']=='компаньон') & (df['total_income'] == 0),'total_income'] = 202417
df.loc[(df['income_type']=='пенсионер') & (df['total_income'] == 0),'total_income'] = 137127
df.loc[(df['income_type']=='предприниматель') & (df['total_income'] == 0),'total_income'] = 499163
df.loc[(df['income_type']=='сотрудник') & (df['total_income'] == 0),'total_income'] = 161380
df.loc[(df['income_type']=='студент') & (df['total_income'] == 0),'total_income'] = 98201

In [28]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,4172.941807,43.293380,0.817236,0.972544,0.080883,1.505129e+05
std,1.381587,5267.296063,12.574584,0.548138,1.420324,0.272661,1.098971e+05
min,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00
25%,0.000000,610.652074,33.000000,1.000000,0.000000,0.000000,8.861283e+04
50%,0.000000,1808.053434,42.000000,1.000000,0.000000,0.000000,1.355147e+05
75%,1.000000,4779.587738,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Еще раз проверим сводные данные. Пропусков нет. Но есть одно отрицательное значение в столбце с количеством детей, что в целом странно. И минимальный возраст равен 0. Рассмотрим эти графы подробнее

In [29]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Есть отрицательные значения, есть очевидно высокие(20) и таких большое количество. Предположим, что это правда, тогда возраст должен быть пенсионный. Сгруппируем по возрасту:

In [30]:
df[df['children'] == 20].groupby('dob_years')['children'].count()

dob_years
0     1
21    1
23    1
24    1
25    1
26    1
27    2
29    2
30    3
31    2
32    2
33    2
34    3
35    2
36    2
37    4
38    1
39    1
40    4
41    2
42    3
43    2
44    2
45    3
46    3
48    1
49    3
50    3
51    1
52    1
53    1
54    1
55    1
56    5
57    1
59    2
60    1
61    1
62    1
64    1
69    1
Name: children, dtype: int64

Как видим, 20 детей встречается и у людей молодого возраста, что в принципе невозможно. Думаю, это опечатка и ввод лишнего 0. Произведем замену значения 20 на 2.

In [31]:
df['children'] =df['children'].replace(20, 2)

Теперь разберемся с отрицательными значениями. Думаю, это также связано с опечатками, скорее всего "-" является как маркировкой. Заменим -1 на 1:

In [32]:
df['children'] =df['children'].replace(-1, 1)
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Теперь посмотрим возраст

In [33]:
df[df['dob_years'] == 0]['dob_years'].count()

101

показателей с 0 значением не так много(0,5%), вместе с тем, у нас не стоит вопрос о зависимости возраста и возвратности кредита. Удалить будет спорным вопросом, так как в других массивах по этим строкам все же может быть важная информация. Оставим, как есть

In [34]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.479721,4172.941807,43.293380,0.817236,0.972544,0.080883,1.505129e+05
std,0.755528,5267.296063,12.574584,0.548138,1.420324,0.272661,1.098971e+05
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00
25%,0.000000,610.652074,33.000000,1.000000,0.000000,0.000000,8.861283e+04
50%,0.000000,1808.053434,42.000000,1.000000,0.000000,0.000000,1.355147e+05
75%,1.000000,4779.587738,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,5.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
category_years      21525 non-null object
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


Пропусков и явно неверных значений нет, данные обработы и готовы к дальнейшим действиям.

### Замена типа данных

Есть дробные значения , заменим на целочисленные.

In [36]:
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)

# Вывод

 Дробно указаны дни работы и зарплата за месяц, за точкой не будет критичного значения, а воспринимтаь целые числа приятнее проще для анализа и подсчета

### Обработка дубликатов

У нас есть столбцы с типом данных str, необходимо такие данные привести к одному регистру и проверить, какая информация в целом там дана. Возможно ли ее преобразовать

Рассмотрим столбец с образованием. Мы не будем рассматривать столб ID, так как это не несет никакой смысловой нагрузки. Проверим, какие есть уникальные значения

In [37]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Видим, что есть дубли из-за регистра. Приведем к одному:

In [38]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Избавились от повторяющихся значений, переходим к семейному положению. Проверяем также уникальные значения:

In [39]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Здесь визуально все в порядке.

ID семейного положения мы также не будем брать во внимание, как и с образованием, перейдем к полу.

In [40]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Здесь есть непонятное значение, посчитаем, сколько таких

In [41]:
df[df['gender'] == 'XNA']['gender'].count()

1

Такoe значениe всего 1, также нам не особо важен пол для анализа, поэтому оставим, как есть

In [42]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

По данным значениям нет никаких замечаний

In [43]:
df['purpose'] = df['purpose'].str.lower()
text = df['purpose'].unique()


Список значительный, но видно, что цели одни и те же написаны разными формулировками, необходимо привести к одной лемме.
Но перед этим закончим работу с дублями и проверим наличие полных дубликатов.

In [44]:
df.duplicated().sum()

71

Возможно, данные вводились несколько раз, поэтому возникли полные дубли(например, перезагрузили страницу, данные уже сохранились, ввели заново-повторная информация). Удалим дубли c перестановкой индексов

In [45]:
df = df.drop_duplicates().reset_index(drop = True)

Все дубликаты удалены.

### Лемматизация

In [46]:
from pymystem3 import Mystem
m = Mystem()
lemmas = m.lemmatize(';'.join(text))
from collections import Counter
print(Counter(lemmas))

Counter({' ': 59, ';': 37, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Как мы видим, у нас появились основные 4 категории : недвижимость, свадьба, автомобиль и образование. Создадим новый столбец с новыми, упорядоченными категориями.

In [47]:
def new_purpose(purpose):
    if 'недвиж' in purpose:
        return 'недвижимость'
    if 'жил' in purpose:
        return 'недвижимость'
    if 'авто' in purpose:
        return 'автомобиль'
    if 'образов' in purpose:
        return 'образование'
    if 'свад' in purpose:
        return 'свадьба'
    return 'другое'
df['new_purpose'] = df['purpose'].apply(new_purpose)


### Вывод

Не смотря на обилие изначально уникальных значений, целей всего 4. Большое "разнообразие" возникло из-за введения целей самостоятельно клиентами. Не хватает ограниченного выбора уже с заранее обозначенным списком целей для оптимизации анализа данных.

### Категоризация данных

Вернемся к столбцу с доходом.Мы не знаем, в какой это валюте и географическое положение заемщиков.Например, это может быть Вьетнамский донг, и тогда 2млн будет не такой высокой зарплатой, как кажется в рублях. Воспользуемся квантилем и разделим клиентов на группы:


In [48]:
df['total_income'].quantile([0.2, 0.4, 0.6, 0.8, 1])

0.2      79166.0
0.4     116407.4
0.6     156425.8
0.8     214606.0
1.0    2265604.0
Name: total_income, dtype: float64

Назовем эти категории "минимальный", "низкий", "средний", "высокий", "сверхвысокий"

In [49]:
def income_group(total_income):
    if total_income < 79166:
        return 'минимальный'
    if total_income <= 116407:
        return 'низкий'
    if total_income <= 156425:
        return 'средний'
    if total_income <= 214606:
        return 'высокий'
    return 'сверхвысокий'
df['income_group'] = df['total_income'].apply(income_group)

In [50]:
df['income_group'].value_counts()

низкий          4292
сверхвысокий    4291
высокий         4291
средний         4290
минимальный     4290
Name: income_group, dtype: int64

# Вывод

Получилось 5 категорий с равным количеством заемщиков - это позволит нам более точно производить анализ , так как данные по "наполняемости" равноценны. Выбрала 5 категорий - разброс значений велик, важно было сделать несколько групп с примерно одинаковой разницей между границами.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

 

In [53]:
df.groupby('children')['debt'].mean()*100

children
0    7.543822
1    9.165808
2    9.492481
3    8.181818
4    9.756098
5    0.000000
Name: debt, dtype: float64

Отсуствует задолженность у категории с 5 детьми, проверим их данные отдельно:

In [54]:
df[df['children'] == 5]['debt']

3977     0
4394     0
7859     0
15787    0
15881    0
16173    0
20387    0
20770    0
21087    0
Name: debt, dtype: int64

Слишком мало данных, чтобы учитывать их значения в общем выводе.

### Вывод

Бездетные клиенты отдают кредиты значительно лучше других, так как наличие иждевенцев - это дополнительная финансовая нагрузка, причем не всегда ожидаемая. При отсуствии иждевенцев можно более точно распределять свои финансы.
Клиенты с 1 и 2 детьми имеют примерно одинаковую статистику возвратностии заема. Продолжая мысль с бездетными клиентами, тут уже появляются люди на иждевении, поэтому, сравнительно с бездетными, вероятность неожидаемых финансовых потерь растет, возвратность кредита ухудшается.
Что касается 3 детей - интересно то, что возвратность лучше, чем у имеющих 1 или 2 детей. Могу предположить, что 1 и 2 ребенок могут появляться в более молодом возрасте, а 3ий уже в зрелом, клиент вырос по карьерной лестнице , увеличился доход и обеспеченность, возможно поэтому и появилось решение о 3 ребенке.
При этом с 4мя детьми самая худшая ситуация. Есть взаимосвязь с возрастом - уже завершение карьеры, ухудшение здоровья и как следствие потеря дохода. 
Исходя из всего выше мы можем сделать вывод, что взаимосвязь возврата кредитов с количеством детей есть.

Перейдем ко второму вопросу : Зависит ли возвратность от семейного положения

In [55]:
df.groupby('family_status')['debt'].mean()*100

family_status
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.347145
женат / замужем          7.545182
Name: debt, dtype: float64

Самый высокий показатель у людей не женатых либо в гражданском браке.Проверим средний возраст:

In [56]:
age = df.groupby('family_status')['dob_years'].mean()
age

family_status
Не женат / не замужем    38.363701
в разводе                45.517992
вдовец / вдова           56.503650
гражданский брак         42.069381
женат / замужем          43.547127
Name: dob_years, dtype: float64

Так и есть, категория не женатых и в гражданском браке самая молодая

# ВЫВОД

По данным можно сдедать вывод, что чем моложе клиент, тем хуже возвраты.  Возможно, это связано более с молодым возрастом - меньше доход(студенты), не понимание отвественности. Да и в целом, вопрос брака - это уже ответственно :) Женатые возвращают кредиты лучше по причине 2 зарабатывающих людей, также люди уже более зрелого возраста, аналогично выше заработок. Думаю, поэтому и разведенные люди на таком же уровне - есть стабильная работа к этому моменту. Что касается вдовцов - возможно, к этому моменту уже помогают дети, плюс уменьшаются потребности (уже есть жилье, машина, например) и прибавляется пенсия. Опять же, беря в расчет также средний возраст.
Подводя итоги, зависимость возврата кредита от семейного статуса есть.

Теперь проверим зависомость от уровня дохода. 

In [57]:
df.groupby('income_group')['debt'].mean()*100

income_group
высокий         8.622699
минимальный     7.762238
низкий          8.457596
сверхвысокий    6.991377
средний         8.741259
Name: debt, dtype: float64

### Вывод

Клиенты со сверхвысоким доходом имеют лучшую статистику возврата по очевидным причинам - высокий доход позволяет распределять средства на все обязательства.
Интересно, что на втором месте по статистике люди с минимальным доходом. Связываю это с тем, что чем меньше доход, тем меньше сумма одобряемого кредита и мелкие суммы проще отдавать.
Высокий, низкий и средеий доход примерно на одном уровне возврата - думаю, здесь у людей растут потребности с появлением более стабильного дохода в виде жилья либо смены авто, суммы и возраст тоже примерно равны.Соответсвенно, причины задолженностей также могут быть аналогичны, отсюда и одинаковая статистика.

Остался последний пункт, выявим тем же способом. Сгруппируем по выделенным целям жилье/недвижимость(искать надо оба слова, но в одну группу), автомобиль,образование, свадьба 

In [58]:
df.groupby('new_purpose')['debt'].mean()*100

new_purpose
автомобиль      9.359034
недвижимость    7.233373
образование     9.220035
свадьба         8.003442
Name: debt, dtype: float64

### Вывод

За жилье и свадьбу лушче возвращают кредит, чем за автомобиль  образование. Как правило, за жилье, если не возвращать кредит, банк забирает недвижимость(залог). А жилье является одной из обязательных потребностей человека, поэтому это показывает на высокую степень возврата.
При этом, машина также в залоге, но таковой потребностью не является.
Образование также можно соотнести с молодым возрастом.
А свадьбы вносятся из средсв подаренных, поэтому суммы возможно не критичные, чтобы их не возвращать.

### Шаг 4. Общий вывод

Перед нами стояла цель выяснить, влияют ли на возврат кредита семейное положение, количество детей и уровень дохода.
Помимо выше упомянутых данных мы знали возраст клиента, его тип занятости, образование , стаж работы и цели кредита.
В данных было много "странностей", таких как отрицательный стаж и количество детей или опыт работы около 1000 лет, были пропуски и дубликаты из-за разного регистра. Все эти ошибки появились из-за "ручного" ввода данных клиента. Если по каждому вопросу ставить ограниченный вариант выбора, невозможность пропустить ввод значения, то данные будут качественнее.
При обработке данных я анализировала происхождение ошибок и находила тот вариант обработки, который меньше всего несет за собой потерю данных.
Я проанализировала данные о задолженности, группируя по разным категориям(возраст,цели, наличие издевенцев и т.д) для более ясной картины. Задолженность по всем категориям не превышает 10% клиентов. Самые стабильные по возврату - это кредиты на недвижимость. Думаю, что людей дисциплинирует залог Банку - есть риск потерять саму цель кредита.
Клиенты, не вступившие в брак, находящиеся в молодом возрасте - к ним нужно особо приглядеться, так как они хуже возвращают ссуду.
Клиенты женатые, в разводе, среднего возраста имеют более стабильный доход, выше уровень отвественности и задолженность имеют реже.
Многодетные семьи от 4 детей также несут максимальные риски.
А бездетные и при этом женатые возвращают кредиты стабильнее.
Отвечая на поставленные вопросы, могу дать ответ, что задолженность зависит от всех 3 пунктов - семейное положение, количество детей и уровень дохода, а также от возраста.
При реорганизации приема данных и опираясь на эти выводы можно учучшить кредитный портфель.
